In [ ]:
!git clone https://github.com/TencentARC/GFPGAN.git
!git clone https://github.com/genclass2022/public.git

In [ ]:
!pip list | grep torch

In [ ]:
!pip install basicsr
!pip install facexlib
!pip install realesrgan

In [ ]:
!wget -N https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P GFPGAN/experiments/pretrained_models

In [ ]:
!ln -s GFPGAN/experiments ./experiments

In [ ]:
!pip install ffmpeg
!pip install imageio-ffmpeg

In [ ]:
from glob import glob
import numpy as np
from PIL import Image
import imageio
import os

frames = imageio.mimread('public/hw02/remastering.mp4', memtest=False)

In [ ]:
def export_video_to_img():
  if not os.path.exists('./input'):
    os.mkdir('./input')
  
  resized_frame = [e[:, 66:-32, :] for e in frames]

  for i, img in enumerate(resized_frame):
      Image.fromarray(resized_frame[i]).save(f'./input/{i}.png', 'png')

export_video_to_img()

In [ ]:
if not os.path.exists('./output'):
  os.mkdir('./output')

!rm -rf /results/*
!python GFPGAN/inference_gfpgan.py -i input/ -o output -v 1.3 -s 4

In [ ]:


def png_paths(root):
    return sorted(glob(f'{root}/*.png'))

def image_loads(paths):
    def load_(p):
        img = Image.open(p).convert('RGB')
        return np.array(img)

    results = [load_(e) for e in paths]

    return results

In [ ]:

filenames = [os.path.basename(x) for x in glob(f'./output/restored_imgs/*.png')]
print(filenames)
names = [int(os.path.splitext(x)[0]) for x in filenames]
names = sorted(names)
print(names)

In [ ]:
sorted_paths = [f'output/restored_imgs/{x}.png' for x in names]
print(sorted_paths)

In [ ]:
def generate_mp4(out_name, images, kwargs):
    writer = imageio.get_writer(out_name , **kwargs)
    for image in images:
        writer.append_data(image)
    writer.close()

output_images = image_loads(sorted_paths)

generate_mp4('results.mp4', output_images, {'fps':30})

